<a href="https://colab.research.google.com/github/A-l-E-v/ML-Engineer/blob/main/Lesson_4_4_(Animations).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import plotly.express as px

## Анимация с помощью plotly.express

Анимация в python выполняется с помощью двух параметров: это *animation_frame*, отвечающий за ось, вдоль которой мы строим анимацию (например, это может быть время, т.е. мы строим график для каждого момента времени, а потом анимируем переходы) и *animation_group*, отвечающий за точки, которые будут подвержены анимированию

In [ ]:
df = px.data.gapminder()
df

In [ ]:
# первая анимация (range_x и range_y должны быть зафиксированы!)

px.scatter(df, x="gdpPercap", y="lifeExp", animation_frame="year", animation_group="country",
           size="pop", color="continent", hover_name="country",
           log_x=True, size_max=55, range_x=[100, 100000], range_y=[25, 90])

In [ ]:
# анимировать можно также столбчатый график

fig = px.bar(df, x="continent", y="pop", color="continent",
             animation_frame="year", animation_group="country", range_y=[0, 4e9])

fig.show()

In [ ]:
# ... и облака точек

fig = px.strip(df, x="pop", y="continent", color="continent", log_x=True, hover_name="country",
               animation_frame="year", animation_group="country", range_x=[1000, 4e9])

fig.update_traces(jitter=0.7)

fig.show()

Текущие ограничения, связанные с анимациями:

* Анимации работают хорошо в тех случаях, когда данные содержатся полностью для каждого объекта из *animation_group* для каждого значения из *animation_frame*, без пропусков. Также категориальные значения, использующиеся для деления по цвету, стилю или рядам/столбцам, для хорошей анимации должны быть постоянны и не меняться от фрейма к фрейму

* Несмотря на то, что анимация поддерживается для многих видов графиков, плавная анимация доступна **только** для типов *scatter* и *bar* (это заметно из примеров выше)

* Plotly не вычисляет объединённый по всем фреймам диапазон значений по осям и цветам автоматически, поэтому во избежание скачков масштаба между фреймами необходимо задать необходимые статические диапазоны для анимации вручную

## Низкоуровневая анимация с помощью plotly.graph_objects()

In [ ]:
import plotly.graph_objects as go
import numpy as np
import pandas as pd

In [ ]:
# создадим анимацию "ползущего отрезка"

fig = go.Figure \
(
    # первый фрейм с отрезком
    data=[go.Scatter(x=[0, 1], y=[0, 1])],
    layout=go.Layout \
    (
        # задаём масштабы и название
        xaxis={"range": [0, 5], "autorange": False},
        yaxis={"range": [0, 5], "autorange": False},
        title="Start",

        # тут задаётся кнопка, которая анимирует объекты - за это отвечает method: "animate"
        # args: [None] отвечает за то, что эта кнопка начала анимации
        updatemenus=[{"type": "buttons",
                      "buttons": [{"label": "Play",
                                   "method": "animate",
                                   "args": [None]}]
                     }]
    ),

    # задаём фреймы
    frames=[go.Frame(data=[go.Scatter(x=[1, 2], y=[1, 2])]),
            go.Frame(data=[go.Scatter(x=[1, 3], y=[1, 3])]),
            go.Frame(data=[go.Scatter(x=[2, 3], y=[2, 3])]),
            go.Frame(data=[go.Scatter(x=[2, 3], y=[2, 1])]),
            go.Frame(data=[go.Scatter(x=[2, 4], y=[2, 0])]),
            go.Frame(data=[go.Scatter(x=[3, 4], y=[1, 0])],
                     layout=go.Layout(title_text="End"))]
)

fig.show()

In [ ]:
# Генерируем овальные данные

t = np.linspace(-np.pi, np.pi, 100)
x_f = lambda x: np.sin(x)
y_f = lambda y: 2*np.cos(y)
x = x_f(t)
y = y_f(t)
x_min = np.min(x) - 1.5
x_max = np.max(x) + 1.5
y_min = np.min(y) - 1.5
y_max = np.max(y) + 1.5

N = 50
steps = np.linspace(-np.pi, np.pi, N)


# Создание рисунка
fig = go.Figure \
(
    # Рисуем голубую линию, которая не будет меняться (достигается путём дублирования)
    data=[go.Scatter(x=x, y=y,
                     mode="lines",
                     line={"width": 2, "color": "cyan"}),
          go.Scatter(x=x, y=y,
                     mode="lines",
                     line={"width": 2, "color": "cyan"})],

    layout=go.Layout \
    (
        # регулируем масштаб с отступами от максимумов и минимумов
        xaxis={"range": [x_min, x_max], "autorange": False, "zeroline": False},
        yaxis={"range": [y_min, y_max], "autorange": False, "zeroline": False},
        title_text="Point movement on an elliptic curve", hovermode="closest",
        # кнопка Play
        updatemenus=[{"type": "buttons",
                      "buttons": [{"label": "Play",
                                   "method": "animate",
                                   "args": [None]}]
                     }]
    ),
    # список фреймов с двигающейся точкой, которая будет анимирована
    frames=[go.Frame(data=[go.Scatter(x=[x_f(step)], y=[y_f(step)],
                                      mode="markers", marker={"color": "orange", "size": 10})]) \
                     for step in steps]
)

fig.show()

In [ ]:
# общая схема построения анимации такова:

# 1. Импортируем данные
# df = pd.read_csv/excel/etc.(...) / px.data.smth()


# 2. Формируем словари для рисунка и анимации
# fig_dict = \
# {
#     "data": [], # первоначальные данные
#     "layout": {}, # список атрибутов графика
#     "frames": [] # список фреймов в формате {"data": [], name: string}
# }

# 2.1 Часть атрибутов (например, layout без слайдера) можно заполнить на месте (необязательно)


# 3. Формируем слайдер
# slider_dict = \
# {
#     ...,
#     "steps": []
# }
# steps - это список шагов слайдера в формате
# step = \
# {
#     "args": \
#     [
#         [it], # этот массив отвечает за значение шага слайдера
#         # а здесь мы прописываем параметры шага и перехода (frame, mode, transition и др.)
#         {
#             ...
#         }
#     ],
#     "label": str(it), # даём шагу название (как правило по значению шага)
#     "method": "animate" # метод для анимации
# }
# steps можно заполнить на месте, либо в шаге 5


# 4. Формируем начальные данные (параметр data)
# data - это список словарей, каждый словарь будет представлять объект,
# который будет первоначально на экране (до запуска слайдера)
# если данные объединены в группу и должны отображаться одним цветом/формой/еtc.,
# им надо дать одно название в поле "name"


# 5. Формируем фреймы (frames)
# frame = {"data": [], "name": it}
# обычно этот словарь формируется так: запускается итератор (it) по значениям шкалы,
# формируются данные аналогично шагу сверху и именуются текущим значением оператора, затем frames.append(frame)
# при этом словари объектов внутри data могут именоваться всё ещё в рамках своей группы
# также на этом шаге удобно формировать step и добавлять его в массив steps, т.к. step завязан на итерации


# 6. Формируем layout (если не был заполнен на шаге 2.1)
# layout = \
# {
#     ...,
#     # для кнопок
#     updatemenus=\
#     [{
#         ...
#         "buttons": \
#         [
#             # для кнопки Play
#             {
#                 "args": [None, {...}], # второй аргумент здесь это параметры фрейма и перехода, как в step
#                 "label": "Play" / "&#9654;",
#                 "method": "animate"
#             },
#             # для кнопки Pause
#             {
#                 "args": [[None], {...}],
#                 "label": "Pause" / "&#x5f0;",
#                 "method": "animate"
#             }
#         ],
#         ...
#     }],
#     ...,
#     # не забываем добавить слайдер
#     sliders=[slider_dict]
# }


# 7. Сборка:

# fig_dict["data"] = data
# fig_dict["layout"] = layout
# fig_dict["frames"] = frames

# fig = go.Figure(fig_dict)
# fig.show()


In [ ]:
px.data.gapminder()

In [ ]:
# здесь мы будем строить анимацию с нуля добавляя слайдеры и кнопки самостоятельно

url = "https://raw.githubusercontent.com/plotly/datasets/master/gapminderDataFiveYear.csv"
dataset = pd.read_csv(url)

years = dataset['year'].unique()
continents = dataset['continent'].unique()

# создаём словарь для будущей анимации
fig_dict = \
{
    "data": [], # первоначальные данные
    "layout": {}, # список атрибутов графика
    "frames": [] # список фреймов в формате {"data": [], name: string}
}

# сразу заполняем поля с атрибутами графика (layout)
fig_dict["layout"]["xaxis"] = {"range": [30, 85], "title": "Life Expectancy"}
fig_dict["layout"]["yaxis"] = {"title": "GDP per Capita", "type": "log"}
fig_dict["layout"]["hovermode"] = "closest" # для всплывающего окна
# для виджетов
fig_dict["layout"]["updatemenus"] = \
[{
    # параметры кнопок
    "buttons": \
    [
        {
            # кнопка пуска анимации: duration для длительностей, redraw для оптимизации
            # redraw работает не для всех типов графиков, но для scatter работает
            "args": [None, {"frame": {"duration": 500, "redraw": False},
                            "fromcurrent": True, "transition": {"duration": 300,
                                                                "easing": "quadratic-in-out"}}],
            "label": "Play",
            "method": "animate"
        },
        {
            # кнопка паузы анимации (ВАЖНО! именно "args": [None], в скобках)
            "args": [[None], {"frame": {"duration": 0, "redraw": False},
                              "mode": "immediate",
                              "transition": {"duration": 0}}],
            "label": "Pause",
            "method": "animate"
        }
    ],
    "direction": "left", # кнопка слева
    "pad": {"r": 10, "t": 87}, # отступы справа и сверху от края окна
    "showactive": False, # не показывать какая кнопка была нажата
    "type": "buttons",
    # далее привязка к координатам
    "x": 0.1,
    "xanchor": "right",
    "y": 0,
    "yanchor": "top"
}]

# параметры слайдера
sliders_dict = \
{
    # начальное положение, считающееся активным
    "active": 0,
    # привязка верхнему левому углу
    "yanchor": "top",
    "xanchor": "left",
    # для надписи, обозначающей в какой части слайдера сейчас находится ползунок
    "currentvalue": \
    {
        "font": {"size": 10},
        "prefix": "Date:",
        "visible": True,
        "xanchor": "right"
    },
    # параметр перехода из положения в положение
    "transition": {"duration": 300, "easing": "cubic-in-out"},
    # отступы сверху и снизу
    "pad": {"b": 10, "t": 50},
    # длина и координаты
    "len": 0.9,
    "x": 0.1,
    "y": 0,
    "steps": []
}

# данные для начала
year = 1952
for continent in continents:
    dataset_by_year = dataset[dataset["year"] == year]
    dataset_by_year_and_cont = dataset_by_year[dataset_by_year["continent"] == continent]

    data_dict = \
    {
        "x": list(dataset_by_year_and_cont["lifeExp"]),
        "y": list(dataset_by_year_and_cont["gdpPercap"]),
        "mode": "markers", # только маркеры
        "text": list(dataset_by_year_and_cont["country"]),
        # регулируем размер маркеров
        "marker": \
        {
            "sizemode": "area", # площадь увеличивается пропорционально числу
            "sizeref": 200000, # относительное значение
            "size": list(dataset_by_year_and_cont["pop"])
        },
        "name": continent # объединяющее имя для группы
    }
    fig_dict["data"].append(data_dict) # добавляем в словарь в атрибут "data"

# создаём фреймы
for year in years:
    # делаем то же самое, что и сверху, но теперь именуем годом
    frame = {"data": [], "name": str(year)}
    for continent in continents:
        dataset_by_year = dataset[dataset["year"] == int(year)]
        dataset_by_year_and_cont = dataset_by_year[dataset_by_year["continent"] == continent]

        data_dict = \
        {
            "x": list(dataset_by_year_and_cont["lifeExp"]),
            "y": list(dataset_by_year_and_cont["gdpPercap"]),
            "mode": "markers",
            "text": list(dataset_by_year_and_cont["country"]),
            "marker": \
            {
                "sizemode": "area",
                "sizeref": 200000,
                "size": list(dataset_by_year_and_cont["pop"])
            },
            "name": continent
        }
        frame["data"].append(data_dict)
    fig_dict["frames"].append(frame)

    # также не забываем делать шаг слайдера
    slider_step = \
    {
        "args": \
        [
            [year],
            {
                "frame": {"duration": 300, "redraw": False}, # длительность и оптимизация фрейма
                "mode": "immediate", # немедленно переходить
                "transition": {"duration": 300} # длительность перехода между фреймами
            }
        ],
        "label": str(year),
        "method": "animate"
    }
    sliders_dict["steps"].append(slider_step)


# добавляем слайдер в атрибуты
fig_dict["layout"]["sliders"] = [sliders_dict]

fig = go.Figure(fig_dict)

fig.show()

In [ ]:
!pip install scikit-image

In [ ]:
# Визуализация данных МРТ

import time
from skimage import io

# считываем данные
vol = io.imread("https://s3.amazonaws.com/assets.datacamp.com/blog_assets/attention-mri.tif")
volume = vol.T
# размеры одного слоя МРТ
r, c = volume[0].shape

# количество фреймов
nb_frames = 68

# здесь мы пропускаем шаг с созданием словаря для рисунка и сразу создаём рисунок с фреймами внутри
# осталось добавить атрибуты layout и data с помощью add_trace()
fig = go.Figure \
(
    frames=\
    [
        go.Frame(data=go.Surface(z=(6.7 - k * 0.1) * np.ones((r, c)),
                                 surfacecolor=np.flipud(volume[67 - k]),
                                 cmin=0, cmax=200),
                 name=str(k)) # фрейм должен быть именован для корректной работы
        for k in range(nb_frames)
    ]
)

# формируем первоначальные данные (добавляем параметр data)
fig.add_trace \
(
    go.Surface(z=6.7 * np.ones((r, c)),
               surfacecolor=np.flipud(volume[67]),
               colorscale='Gray', cmin=0, cmax=200,
               colorbar={"thickness": 20, "ticklen": 4})
)


# функция для формирования словарей фреймов
def frame_args(duration):
    return \
{
        "frame": {"duration": duration},
        "mode": "immediate",
        "fromcurrent": True,
        "transition": {"duration": duration, "easing": "linear"},
}

# формируем слайдер
sliders = \
[
    {
        "pad": {"b": 10, "t": 60},
        "len": 0.9,
        "x": 0.1,
        "y": 0,
        "steps": \
        [
            {
                "args": [[f.name], frame_args(0)],
                "label": str(k),
                "method": "animate",
            }
            for k, f in enumerate(fig.frames)
        ],
    }
]

# добавляем параметр layout
fig.update_layout\
(
    # подпись и размеры 3d графика
    title='Slices in volumetric data',
    width=600,
    height=600,
    scene=\
    {
        "zaxis": {"range": [-0.1, 6.8], "autorange": False},
        "aspectratio": {'x': 1, 'y': 1, 'z': 1}
    },
    # кнопки
    updatemenus=\
    [{
        "buttons": \
        [
            # кнопка play
            {
                "args": [None, frame_args(50)],
                "label": "&#x25b6;", # play symbol
                "method": "animate",
            },
            # кнопка pause
            {
                "args": [[None], frame_args(0)],
                "label": "&#x5f0;", # pause symbol
                "method": "animate",
            },
        ],
        "direction": "left",
        "pad": {"r": 10, "t": 70},
        "type": "buttons",
        "x": 0.1,
        "y": 0,
    }],
    # добавляем полученный слайдер в layout
    sliders=sliders
)

fig.show()

## Дополнительные материалы

[Документация по слайдерам](https://plotly.com/python/reference/layout/sliders/)

[Документация по updatemenus](https://plotly.com/python/reference/layout/updatemenus/)